In [567]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ρωμανός\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [568]:
df = pd.read_csv(r'onion-or-not.csv',encoding='utf8')
pd.options.display.width = None

df.head(10)

,text,label
0,Entire Facebook Staff Laughs As Man Tightens Privacy Settings,1
1,Muslim Woman Denied Soda Can for Fear She Could Use It as a Weapon,0
2,Bold Move: Hulu Has Announced That They’re Gonna Go Ahead And Reboot ‘Shrill’ While It’s Still On Since You Idiots Will Watch Anything,1
3,Despondent Jeff Bezos Realizes He’ll Have To Work For 9 Seconds To Earn Back Money He Lost In Divorce,1
4,"For men looking for great single women, online dating offers a viable solution to the otherwise frustrating task of finding long-term love. A0rNsK5",1
5,Kim Jong-Un Wonders If Nuclear Threats Distracting Him From Real Goal Of Starving Citizenry,1
6,"Omaha dad finds pot brownies, eats 4 of them, says mean things to cat",0
7,"Pokémon Go player stabbed, keeps playing",0
8,Job Placement Service Helps Students Who Fail Out Of Dad’s Alma Mater Find Work At Dad’s Company,1
9,Idiot Zoo Animal With Zero Predators Still Protective Of Young,1


Χωρίζουμε τους τίτλους σε λέξεις, δημιουργώντας ένα διάνυσμα λέξεων


In [569]:
df["tokens1"] = df["text"].apply(nltk.word_tokenize) #with punctuation
tokenizer = RegexpTokenizer(r'\w+')
df['tokens'] = df["text"].apply(tokenizer.tokenize) #without punctuation

df.head(10)

,text,label,tokens1,tokens
0,Entire Facebook Staff Laughs As Man Tightens Privacy Settings,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]","[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]"
1,Muslim Woman Denied Soda Can for Fear She Could Use It as a Weapon,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]","[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]"
2,Bold Move: Hulu Has Announced That They’re Gonna Go Ahead And Reboot ‘Shrill’ While It’s Still On Since You Idiots Will Watch Anything,1,"[Bold, Move, :, Hulu, Has, Announced, That, They, ’, re, Gon, na, Go, Ahead, And, Reboot, ‘, Shrill, ’, While, It, ’, s, Still, On, Since, You, Idiots, Will, Watch, Anything]","[Bold, Move, Hulu, Has, Announced, That, They, re, Gonna, Go, Ahead, And, Reboot, Shrill, While, It, s, Still, On, Since, You, Idiots, Will, Watch, Anything]"
3,Despondent Jeff Bezos Realizes He’ll Have To Work For 9 Seconds To Earn Back Money He Lost In Divorce,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]","[Despondent, Jeff, Bezos, Realizes, He, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]"
4,"For men looking for great single women, online dating offers a viable solution to the otherwise frustrating task of finding long-term love. A0rNsK5",1,"[For, men, looking, for, great, single, women, ,, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long-term, love, ., A0rNsK5]","[For, men, looking, for, great, single, women, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long, term, love, A0rNsK5]"
5,Kim Jong-Un Wonders If Nuclear Threats Distracting Him From Real Goal Of Starving Citizenry,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]","[Kim, Jong, Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]"
6,"Omaha dad finds pot brownies, eats 4 of them, says mean things to cat",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4, of, them, ,, says, mean, things, to, cat]","[Omaha, dad, finds, pot, brownies, eats, 4, of, them, says, mean, things, to, cat]"
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]"
8,Job Placement Service Helps Students Who Fail Out Of Dad’s Alma Mater Find Work At Dad’s Company,1,"[Job, Placement, Service, Helps, Students, Who, Fail, Out, Of, Dad, ’, s, Alma, Mater, Find, Work, At, Dad, ’, s, Company]","[Job, Placement, Service, Helps, Students, Who, Fail, Out, Of, Dad, s, Alma, Mater, Find, Work, At, Dad, s, Company]"
9,Idiot Zoo Animal With Zero Predators Still Protective Of Young,1,"[Idiot, Zoo, Animal, With, Zero, Predators, Still, Protective, Of, Young]","[Idiot, Zoo, Animal, With, Zero, Predators, Still, Protective, Of, Young]"


2.	Από τις λέξεις αφαιρούμε τις καταλήξεις τους, κρατώντας μόνο το θέμα τους (stemming)


In [570]:
stemmer = PorterStemmer()
df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
#df = df.drop(columns=['unstemmed']) # Get rid of the unstemmed column.
df.head(10) # Print dataframe.

,text,label,tokens1,tokens,stemmed
0,Entire Facebook Staff Laughs As Man Tightens Privacy Settings,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]","[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]","[entir, facebook, staff, laugh, As, man, tighten, privaci, set]"
1,Muslim Woman Denied Soda Can for Fear She Could Use It as a Weapon,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]","[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]","[muslim, woman, deni, soda, can, for, fear, she, could, use, It, as, a, weapon]"
2,Bold Move: Hulu Has Announced That They’re Gonna Go Ahead And Reboot ‘Shrill’ While It’s Still On Since You Idiots Will Watch Anything,1,"[Bold, Move, :, Hulu, Has, Announced, That, They, ’, re, Gon, na, Go, Ahead, And, Reboot, ‘, Shrill, ’, While, It, ’, s, Still, On, Since, You, Idiots, Will, Watch, Anything]","[Bold, Move, Hulu, Has, Announced, That, They, re, Gonna, Go, Ahead, And, Reboot, Shrill, While, It, s, Still, On, Since, You, Idiots, Will, Watch, Anything]","[bold, move, hulu, ha, announc, that, they, re, gonna, Go, ahead, and, reboot, shrill, while, It, s, still, On, sinc, you, idiot, will, watch, anyth]"
3,Despondent Jeff Bezos Realizes He’ll Have To Work For 9 Seconds To Earn Back Money He Lost In Divorce,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]","[Despondent, Jeff, Bezos, Realizes, He, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]","[despond, jeff, bezo, realiz, He, ll, have, To, work, for, 9, second, To, earn, back, money, He, lost, In, divorc]"
4,"For men looking for great single women, online dating offers a viable solution to the otherwise frustrating task of finding long-term love. A0rNsK5",1,"[For, men, looking, for, great, single, women, ,, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long-term, love, ., A0rNsK5]","[For, men, looking, for, great, single, women, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long, term, love, A0rNsK5]","[for, men, look, for, great, singl, women, onlin, date, offer, a, viabl, solut, to, the, otherwis, frustrat, task, of, find, long, term, love, a0rnsk5]"
5,Kim Jong-Un Wonders If Nuclear Threats Distracting Him From Real Goal Of Starving Citizenry,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]","[Kim, Jong, Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]","[kim, jong, Un, wonder, If, nuclear, threat, distract, him, from, real, goal, Of, starv, citizenri]"
6,"Omaha dad finds pot brownies, eats 4 of them, says mean things to cat",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4, of, them, ,, says, mean, things, to, cat]","[Omaha, dad, finds, pot, brownies, eats, 4, of, them, says, mean, things, to, cat]","[omaha, dad, find, pot, browni, eat, 4, of, them, say, mean, thing, to, cat]"
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]","[pokémon, Go, player, stab, keep, play]"
8,Job Placement Service Helps Students Who Fail Out Of Dad’s Alma Mater Find Work At Dad’s Company,1,"[Job, Placement, Service, Helps, Students, Who, Fail, Out, Of, Dad, ’, s, Alma, Mater, Find, Work, At, Dad, ’, s, Company]","[Job, Placement, Service, Helps, Students, Who, Fail, Out, Of, Dad, s, Alma, Mater, Find, Work, At, Dad, s, Company]","[job, placement, servic, help, student, who, fail, out, Of, dad, s, alma, mater, find, work, At, dad, s, compani]"
9,Idiot Zoo Animal With Zero Predators Still Protective Of Young,1,"[Idiot, Zoo, Animal, With, Zero, Predators, Still, Protective, Of, Young]","[Idiot, Zoo, Animal, With, Zero, Predators, Still, Protect

Αφαιρούμε από την συλλογή εκείνες τις λέξεις που είναι αρκετά κοινές και δεν προσφέρουν πληροφορία (stopwords removal)

In [571]:
#st = [item.lower() for item in df['stemmed'].split()]
# df['stemmed'] = df['stemmed'].astype(str).str.lower().str.split()
df['without_stopwords'] = df['stemmed'].apply(lambda x: [item.lower() for item in x if item.lower() not in stop])
df.head(10)

,text,label,tokens1,tokens,stemmed,without_stopwords
0,Entire Facebook Staff Laughs As Man Tightens Privacy Settings,1,"[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]","[Entire, Facebook, Staff, Laughs, As, Man, Tightens, Privacy, Settings]","[entir, facebook, staff, laugh, As, man, tighten, privaci, set]","[entir, facebook, staff, laugh, man, tighten, privaci, set]"
1,Muslim Woman Denied Soda Can for Fear She Could Use It as a Weapon,0,"[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]","[Muslim, Woman, Denied, Soda, Can, for, Fear, She, Could, Use, It, as, a, Weapon]","[muslim, woman, deni, soda, can, for, fear, she, could, use, It, as, a, weapon]","[muslim, woman, deni, soda, fear, could, use, weapon]"
2,Bold Move: Hulu Has Announced That They’re Gonna Go Ahead And Reboot ‘Shrill’ While It’s Still On Since You Idiots Will Watch Anything,1,"[Bold, Move, :, Hulu, Has, Announced, That, They, ’, re, Gon, na, Go, Ahead, And, Reboot, ‘, Shrill, ’, While, It, ’, s, Still, On, Since, You, Idiots, Will, Watch, Anything]","[Bold, Move, Hulu, Has, Announced, That, They, re, Gonna, Go, Ahead, And, Reboot, Shrill, While, It, s, Still, On, Since, You, Idiots, Will, Watch, Anything]","[bold, move, hulu, ha, announc, that, they, re, gonna, Go, ahead, and, reboot, shrill, while, It, s, still, On, sinc, you, idiot, will, watch, anyth]","[bold, move, hulu, ha, announc, gonna, go, ahead, reboot, shrill, still, sinc, idiot, watch, anyth]"
3,Despondent Jeff Bezos Realizes He’ll Have To Work For 9 Seconds To Earn Back Money He Lost In Divorce,1,"[Despondent, Jeff, Bezos, Realizes, He, ’, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]","[Despondent, Jeff, Bezos, Realizes, He, ll, Have, To, Work, For, 9, Seconds, To, Earn, Back, Money, He, Lost, In, Divorce]","[despond, jeff, bezo, realiz, He, ll, have, To, work, for, 9, second, To, earn, back, money, He, lost, In, divorc]","[despond, jeff, bezo, realiz, work, 9, second, earn, back, money, lost, divorc]"
4,"For men looking for great single women, online dating offers a viable solution to the otherwise frustrating task of finding long-term love. A0rNsK5",1,"[For, men, looking, for, great, single, women, ,, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long-term, love, ., A0rNsK5]","[For, men, looking, for, great, single, women, online, dating, offers, a, viable, solution, to, the, otherwise, frustrating, task, of, finding, long, term, love, A0rNsK5]","[for, men, look, for, great, singl, women, onlin, date, offer, a, viabl, solut, to, the, otherwis, frustrat, task, of, find, long, term, love, a0rnsk5]","[men, look, great, singl, women, onlin, date, offer, viabl, solut, otherwis, frustrat, task, find, long, term, love, a0rnsk5]"
5,Kim Jong-Un Wonders If Nuclear Threats Distracting Him From Real Goal Of Starving Citizenry,1,"[Kim, Jong-Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]","[Kim, Jong, Un, Wonders, If, Nuclear, Threats, Distracting, Him, From, Real, Goal, Of, Starving, Citizenry]","[kim, jong, Un, wonder, If, nuclear, threat, distract, him, from, real, goal, Of, starv, citizenri]","[kim, jong, un, wonder, nuclear, threat, distract, real, goal, starv, citizenri]"
6,"Omaha dad finds pot brownies, eats 4 of them, says mean things to cat",0,"[Omaha, dad, finds, pot, brownies, ,, eats, 4, of, them, ,, says, mean, things, to, cat]","[Omaha, dad, finds, pot, brownies, eats, 4, of, them, says, mean, things, to, cat]","[omaha, dad, find, pot, browni, eat, 4, of, them, say, mean, thing, to, cat]","[omaha, dad, find, pot, browni, eat, 4, say, mean, thing, cat]"
7,"Pokémon Go player stabbed, keeps playing",0,"[Pokémon, Go, player, stabbed, ,, keeps, playing]","[Pokémon, Go, player, stabbed, keeps, playing]","[pokémon, Go, player, stab, keep, play]","[pokémon, go, player, stab, keep, play]"
8,Job Placement Service H

αναθέτουμε ως βάρος την τιμή tf-idf

In [572]:
tf = TfidfVectorizer(lowercase='false', analyzer='word')
st = df['without_stopwords'].apply(lambda x: ' '.join(x))
#dl = df['without_stopwords'].values.ravel().tolist()
#df['without_stopwords'] = [" ".join(review) for review in df['without_stopwords'].values]
#response = df['without_stopwords'].apply(tf.fit for x in df["without_stopwords"] )
response = tf.fit_transform(st)
re = np.array(response)
print(re)
#tf.vocabulary_

  (0, 13069)	0.30924352058001103
  (0, 11532)	0.4038006611301081
  (0, 14781)	0.5072544198071636
  (0, 9035)	0.1691140585248853
  (0, 8460)	0.3920273973386792
  (0, 13880)	0.35262233959490963
  (0, 5385)	0.2832299661866386
  (0, 5114)	0.3117044296347044
  (1, 15979)	0.400093851507405
  (1, 15567)	0.2659197854105327
  (1, 3628)	0.3025678760170136
  (1, 5504)	0.3818448434646714
  (1, 13610)	0.4565533569455909
  (1, 4200)	0.3601554234253247
  (1, 16216)	0.23435496864771768
  (1, 9853)	0.3725367438848746
  (2, 1159)	0.24411062532220687
  (2, 15940)	0.1991716368233393
  (2, 7368)	0.2726168304029761
  (2, 13344)	0.24251698633926413
  (2, 14015)	0.19753548579753888
  (2, 13273)	0.36706688407421084
  (2, 11982)	0.310222064605211
  (2, 843)	0.2691399593013618
  (2, 6346)	0.17897387591761013
  :	:
  (23996, 2819)	0.27446125275141814
  (23996, 5648)	0.2255999313831728
  (23996, 13942)	0.29017605329501134
  (23996, 4017)	0.20579527391561858
  (23996, 16220)	0.22033063330798958
  (23996, 15567)	0.2